<img src="https://github.com/hernancontigiani/ceia_memorias_especializacion/raw/master/Figures/logoFIUBA.jpg" width="500" align="center">


# Procesamiento de lenguaje natural
## Custom embedddings con Gensim



### Objetivo
El objetivo es utilizar documentos / corpus para crear embeddings de palabras basado en ese contexto. Se utilizará canciones de bandas para generar los embeddings, es decir, que los vectores tendrán la forma en función de como esa banda haya utilizado las palabras en sus canciones.

### Datos
Utilizaremos como dataset canciones de bandas de habla inglesa.

### Consigna del desafío 2

**Cada experimento realizado debe estar acompañado de una explicación o interpretación de lo observado**

Recuerden que su notebook de entrega debe poder correrse de inicio a fin sin la aparición de errores.

# 1 Crear sus propios vectores con Gensim basado en lo visto en clase con otro artista del dataset Songs.

Analizamos la cantidad e canciones de cad artisdta para definir con cual trabajar

In [6]:
# Descargar la carpeta de dataset
import os
import platform
if os.access('./songs_dataset', os.F_OK) is False:
    if os.access('songs_dataset.zip', os.F_OK) is False:
        if platform.system() == 'Windows':
            !curl https://raw.githubusercontent.com/FIUBA-Posgrado-Inteligencia-Artificial/procesamiento_lenguaje_natural/main/datasets/songs_dataset.zip -o songs_dataset.zip
        else:
            !wget songs_dataset.zip https://github.com/FIUBA-Posgrado-Inteligencia-Artificial/procesamiento_lenguaje_natural/raw/main/datasets/songs_dataset.zip
    !unzip -q songs_dataset.zip
else:
    print("El dataset ya se encuentra descargado")





El dataset ya se encuentra descargado


In [7]:
import os, re, glob, random
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import ENGLISH_STOP_WORDS
from gensim.models import Word2Vec
from collections import Counter

# config
DATA_DIR = "songs_dataset"
ARTIST = "michael jackson"
EMBED_DIM, WINDOW, MIN_COUNT, EPOCHS, NEGATIVE, SEED = 100, 5, 2, 20, 10, 42
np.random.seed(SEED); random.seed(SEED)

def read_artist_file(path):
    """Devuelve una lista de 'canciones' a partir de un .txt (bloques por líneas en blanco)."""
    with open(path, "r", encoding="utf-8", errors="ignore") as f:
        txt = f.read().replace("\r\n","\n").replace("\r","\n").strip()
    chunks = [c.strip() for c in re.split(r"\n{2,}", txt) if c.strip()]
    songs = [c for c in chunks if len(c.split()) > 15]
    return songs or [txt]

def filename_to_artist(fname):
    base = os.path.basename(fname)
    return os.path.splitext(base)[0].replace("_"," ").strip().lower()

# armar corpus desde .txt
rows = []
for path in sorted(glob.glob(os.path.join(DATA_DIR, "*.txt"))):
    artist = filename_to_artist(path)
    for i, song in enumerate(read_artist_file(path), start=1):
        rows.append({"artist": artist, "song_id": i, "lyrics": song})
df_raw = pd.DataFrame(rows)

# filtrar artista
norm = lambda s: re.sub(r"\s+"," ", str(s).strip().lower())
df_raw["_artist_norm"] = df_raw["artist"].map(norm)
mask = df_raw["_artist_norm"].str.contains(r"\bmichael\b.*\bjackson\b", regex=True)
mj = df_raw[mask].dropna(subset=["lyrics"]).copy()
mj = mj[mj["lyrics"].astype(str).str.strip().ne("")].reset_index(drop=True)
print(f"Canciones de {ARTIST}: {len(mj)}")

# tokenización mínima
STOP = set(ENGLISH_STOP_WORDS) | {"chorus","verse"}
TOKEN_RE = re.compile(r"[a-zA-Z][a-zA-Z']+")
def tokenize(text):
    t = text.lower()
    toks = [w.strip("'") for w in TOKEN_RE.findall(t)]
    return [w for w in toks if len(w) > 1 and w not in STOP]

mj["tokens"] = mj["lyrics"].astype(str).map(tokenize)
sentences = mj["tokens"].tolist()
print(f"#canciones={len(sentences)}  #tokens={sum(len(s) for s in sentences)}  vocab~={len({t for s in sentences for t in s})}")

# entrenar CBOW y Skip-gram
models = {}
for sg in (0,1):
    name = "CBOW" if sg==0 else "SkipGram"
    m = Word2Vec(
        sentences=sentences,
        vector_size=EMBED_DIM,
        window=WINDOW,
        min_count=MIN_COUNT,
        negative=NEGATIVE,
        workers=1,
        sg=sg,
        seed=SEED,
        epochs=EPOCHS
    )
    models[name] = m
    print(f"[{name}] vocab={len(m.wv)}")



Canciones de michael jackson: 1231
#canciones=1231  #tokens=21958  vocab~=2860
[CBOW] vocab=1525
[SkipGram] vocab=1525


 se utilizaron 1 231 canciones de *Michael Jackson*, con unas 21 958 palabras y un vocabulario efectivo de 1 525 términos. La cantidad de datos es suficiente para obtener embeddings representativos del estilo y vocabulario del artista.

 Se usó `MIN_COUNT=2` para filtrar ruido y palabras únicas, que suelen ser irrelevantes. Cien dimensiones (`EMBED_DIM=100`) se entgiende que deberían ser suficientes para este tamaño de vocabulario sin causar sobreajuste. Una ventana de 5 (`WINDOW=5`) captura bien el contexto local de las frases cortas de las canciones. Finalmente, debido al tamaño reducido del corpus, se usaron más *epochs* (`EPOCHS=20`) y más ejemplos negativos (`NEGATIVE=10`) para asegurar que el modelo pudiera converger y aprender las relaciones de manera precisa.

Se seleccionaron estos hiperparámetros buscando un equilibrio para nuestro corpus específico de 1 525 términos.

# 2 Elegir términos de interés y buscar términos más similares y menos similares.

In [13]:
from collections import Counter

# términos a evaluar (usa frecuentes del propio corpus para evitar OOV)
counts = Counter([t for s in sentences for t in s])
common = [w for (w,_) in counts.most_common(300)]
probe = [w for w in ["love","night","dance","girl","heart","feel"] if w in common] or common[:6]

def mas_similares(model, w, n=8):
    return model.wv.most_similar(positive=[w], topn=n) if w in model.wv else None

def menos_similares_profe(model, w, n=8):
    return model.wv.most_similar(negative=[w], topn=n) if w in model.wv else None

for name, m in models.items():
    print(f"\n=== {name} ===")
    for w in probe[:5]:
        top = mas_similares(m, w, n=6)
        low = menos_similares_profe(m, w, n=6)
        if top is None:
            print(f"{w}: OOV"); continue
        print(f"{w} · más similares:   " + ", ".join(f"{t}({s:.2f})" for t,s in top))
        print(f"{w} · 'menos' (neg=-w): " + ", ".join(f"{t}({s:.2f})" for t,s in low))



=== CBOW ===
love · más similares:   beautiful(0.82), greatest(0.81), want(0.78), lady(0.78), makes(0.78), liberian(0.78)
love · 'menos' (neg=-w): pah(0.45), hoo(0.03), doin(0.02), dirty(0.00), where's(-0.04), ticket(-0.05)
night · más similares:   workin(0.92), day(0.87), sugar(0.86), rock(0.84), thriller(0.83), sunlight(0.82)
night · 'menos' (neg=-w): pah(0.61), gone(-0.02), midway(-0.02), stop(-0.03), mercy(-0.04), everybody(-0.05)
dance · más similares:   floor(0.95), blood(0.93), let's(0.83), especially(0.82), inches(0.82), shake(0.81)
dance · 'menos' (neg=-w): pah(0.45), hoo(-0.06), gone(-0.07), she's(-0.07), oh(-0.10), life(-0.11)
girl · más similares:   liberian(0.82), ooh(0.80), love(0.77), bout(0.76), nest(0.75), beautiful(0.75)
girl · 'menos' (neg=-w): pah(0.33), hoo(0.03), tu(-0.02), dirty(-0.04), je(-0.04), amor(-0.07)
heart · más similares:   far(0.91), circus(0.90), pieces(0.90), carousel(0.87), apart(0.86), magical(0.85)
heart · 'menos' (neg=-w): pah(0.46), ain't(-0.07

  - **CBOW** generó asociaciones muy consistentes con el contexto general de las letras.  
    Los vecinos más cercanos muestran relaciones claras entre palabras comunes del lenguaje afectivo y rítmico de Michael Jackson:  
    *love → beautiful, greatest, want, lady*  
    *night → day, rock, thriller*  
    *dance → floor, shake, blood*  
    *girl → liberian, ooh, love*  
    *heart → pieces, far, apart*.  
    En contraste, los términos menos similares (`dirty`, `where's`, `pah`, `hoo`) suelen ser onomatopeyas o palabras sin carga semántica fuerte, lo que demuestra que CBOW prioriza las co-ocurrencias más frecuentes y desestima los términos aislados o performativos.
  
  - **Skip-Gram** mostró un comportamiento más disperso y específico.  
    Los vecinos cercanos presentan una semántica más variada y matizada:  
    *love → greatest, earth, romancing*  
    *dance → simple, disco, floor*  
    *girl → persuasive, liberian, precious*  
    *heart → child's, opened, apart*.  
    Los términos menos similares (`law`, `mother`, `dying`, `black`, `tired`) reflejan conceptos ajenos al tono romántico y festivo predominante del corpus, lo que evidencia que Skip-Gram distingue mejor los temas y separa contextos contrastantes.


  Ambos modelos captan de manera coherente el universo léxico del artista, aunque con diferencias notables:
  - **CBOW** privilegia las relaciones léxicas más comunes y estables, representando el eje *amor-baile-noche* típico de las letras pop.  
  - **Skip-Gram** resalta co-ocurrencias más específicas y contextuales, diferenciando mejor los registros emocionales y expresivos.  


  Los resultados confirman la teoría vista en clase: **CBOW** es más estable y generalista en corpus de tamaño medio, mientras que **Skip-Gram** capta mejor las relaciones contextuales poco frecuentes.  
  Las palabras más y menos similares muestran que ambos modelos aprendieron correctamente los patrones temáticos dominantes del vocabulario de Michael Jackson.


#  3 Realizar una reduccion de dimensionalidad a los embeddings, llevándolos a 2 dimensiones. Graficar los embeddings proyectados y seleccionar una cantidad de términos (variable MAX_WORDS) de forma tal que la visualización sea adecuada.

In [11]:
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
import plotly.express as px

MAX_WORDS = 400
SEED = 42
ANNOTATE_TOP = 150  # cuántas etiquetas mostrar (hover siempre muestra)

def tsne_df(model, max_words=MAX_WORDS, seed=SEED):
    vocab = list(model.wv.key_to_index)[:max_words]
    X = np.vstack([model.wv[w] for w in vocab])
    Xp = PCA(n_components=min(50, X.shape[1]), random_state=seed).fit_transform(X)
    X2 = TSNE(n_components=2, init="pca", random_state=seed, learning_rate="auto").fit_transform(Xp)
    df = pd.DataFrame({"word": vocab, "x": X2[:,0], "y": X2[:,1]})
    df["label"] = df["word"].where(df.index < ANNOTATE_TOP, "")  # solo primeras N con texto fijo
    return df

def plot_interactive(df, title):
    fig = px.scatter(
        df, x="x", y="y",
        text="label", hover_name="word",
        width=900, height=700
    )
    fig.update_traces(textposition="top center", marker=dict(size=6, opacity=0.7))
    fig.update_layout(title=title, dragmode="pan")
    fig.show()

# Usar con tus modelos entrenados:
df_cbow = tsne_df(models["CBOW"], max_words=MAX_WORDS, seed=SEED)
plot_interactive(df_cbow, f"t-SNE — CBOW ({len(df_cbow)} palabras)")

df_sg = tsne_df(models["SkipGram"], max_words=MAX_WORDS, seed=SEED)
plot_interactive(df_sg, f"t-SNE — SkipGram ({len(df_sg)} palabras)")




se aplicó una reducción de dimensionalidad **PCA → t-SNE** para proyectar los embeddings entrenados en dos dimensiones, permitiendo observar agrupamientos semánticos de palabras.

se seleccionaron 400 palabras para lograr un equilibrio entre **detalle y legibilidad**.  
  - Con menos de 200 puntos la gráfica perdía contexto semántico.  
  - Con más de 600, la densidad de etiquetas dificultaba la lectura.  
  Por ello, se hizo este ejercicio con 400 entendiendolo como una  cantidad adecuada para visualizar la estructura global del vocabulario sin saturar el plano.

- **Distribución CBOW:** la proyección muestra un espacio más **compacto y homogéneo**, con menor dispersión entre términos frecuentes. Las palabras se agrupan en zonas generales (amor, baile, noche), reflejando el carácter promedio y estable de CBOW en corpus limitados.

- **Distribución Skip-Gram:** presenta una **dispersión mayor** y clústeres más definidos, donde se observan grupos relacionados con emociones (*love, heart, pain*), acción o movimiento (*dance, rock, floor*), y referencias personales (*michael, girl, man*). Esto evidencia que Skip-Gram capturó relaciones más contextuales y específicas, coherente con su naturaleza predictiva palabra-a-contexto.

ambos modelos representan correctamente la semántica del corpus de *Michael Jackson*. CBOW concentra los términos más comunes del lenguaje de sus letras, mientras que Skip-Gram distingue mejor las temáticas dominantes —amor, baile, fama, soledad— que caracterizan su obra.



# 4 Inspeccionar el grafico y buscar pequeños grupos de palabras que puedan formarse. Interpretarlos e intentar obtener conclusiones. En lo posible, acompañar los grupos de palabras con capturas (y pegarlas en celdas de texto)

In [10]:
from plotly.subplots import make_subplots
import plotly.graph_objects as go

def present(df, words):
    s = set(df['word'])
    keep = [w for w in words if w in s]
    miss = [w for w in words if w not in s]
    if miss:
        print("[WARN] no están en el t-SNE:", miss)
    return keep

def expand_with_neighbors(model, seeds, k=5):
    exp = set(seeds)
    for w in seeds:
        if w in model.wv:
            exp.update([t for t,_ in model.wv.most_similar(w, topn=k)])
        else:
            print(f"[WARN] '{w}' no existe en el vocabulario del modelo.")
    return list(exp)

def auto_limits(df, words, pad=1.5):
    sub = df[df.word.isin(words)]
    if sub.empty:
        return None, None
    xmin, xmax = sub['x'].min(), sub['x'].max()
    ymin, ymax = sub['y'].min(), sub['y'].max()
    return [xmin-pad, xmax+pad], [ymin-pad, ymax+pad]

def add_panel(fig, r, c, df, title, words, bg_opacity=0.25):
    base = go.Scatter(
        x=df["x"], y=df["y"], mode="markers",
        marker=dict(size=4, color="lightgray", opacity=bg_opacity),
        hovertext=df["word"], hoverinfo="text", showlegend=False
    )
    fig.add_trace(base, row=r, col=c)

    sub = df[df.word.isin(words)]
    hi = go.Scatter(
        x=sub["x"], y=sub["y"],
        mode="markers+text",
        text=sub["word"], textposition="top center",
        marker=dict(size=10, color="red"),
        hovertext=sub["word"], hoverinfo="text", showlegend=False
    )
    fig.add_trace(hi, row=r, col=c)

    xr, yr = auto_limits(df, words)
    if xr and yr:
        fig.update_xaxes(range=xr, row=r, col=c)
        fig.update_yaxes(range=yr, row=r, col=c)
    fig.update_annotations(font_size=12)

# semillas por cluster
seeds_emocional = ["love","heart","soul","pain","lonely","feel","forever"]
seeds_baile     = ["dance","floor","shake","beat","rock"]
seeds_social    = ["world","people","believe","michael","change"]
seeds_cbow_mix  = ["love","girl","baby","time","night","remember"]


emo_words  = present(df_sg, seeds_emocional)
emo_words  = expand_with_neighbors(models["SkipGram"], emo_words, k=3)

dance_words = present(df_sg, seeds_baile)
dance_words = expand_with_neighbors(models["SkipGram"], dance_words, k=3)

social_words = present(df_sg, seeds_social)
social_words = expand_with_neighbors(models["SkipGram"], social_words, k=3)

cbow_words = present(df_cbow, seeds_cbow_mix)
cbow_words = expand_with_neighbors(models["CBOW"], cbow_words, k=3)

fig = make_subplots(
    rows=2, cols=2,
    subplot_titles=[
        "SkipGram — Emocional",
        "SkipGram — Baile",
        "SkipGram — Social / Reflexivo",
        "CBOW — Amor / Temporal"
    ],
    horizontal_spacing=0.06, vertical_spacing=0.12
)

add_panel(fig, 1, 1, df_sg,  "SkipGram — Emocional", emo_words)
add_panel(fig, 1, 2, df_sg,  "SkipGram — Baile",     dance_words)
add_panel(fig, 2, 1, df_sg,  "SkipGram — Social",    social_words)
add_panel(fig, 2, 2, df_cbow,"CBOW — Mixto",         cbow_words)

fig.update_layout(
    title="Agrupamientos semánticos detectados — Michael Jackson",
    width=1200, height=900, showlegend=False
)
fig.show()


**CBOW**
- El modelo forma grupos **semánticos amplios y estables**, reflejando su naturaleza promedio de contexto.  
  - **Grupo emocional:** `love – girl – baby – want – feel` muestra coherencia en torno al eje afectivo y relacional, muy presente en el lenguaje romántico de Michael Jackson.  
  - **Grupo rítmico:** `dance – floor – rock – shake – beat` asocia términos ligados al movimiento y la energía del baile.  
  - **Grupo narrativo/temporal:** `night – time – remember – workin` reúne palabras temporales o introspectivas, asociadas a la rutina y al paso del tiempo.  
- En general, CBOW concentra los términos frecuentes y suaviza las fronteras entre temas, produciendo un espacio más homogéneo.

**Skip-Gram**
- Skip-Gram genera agrupamientos **más definidos y contextuales**, lo que evidencia una mejor captura de co-ocurrencias específicas.
  - **Cluster emocional:** `love – heart – soul – pain – feel – forever` representa el eje de emociones y vínculos.  
  - **Cluster de performance:** `dance – shake – beat – rock – floor` vincula expresiones relacionadas al ritmo y la actuación.  
  - **Cluster reflexivo/social:** `world – people – believe – change – michael` concentra los términos asociados a la visión humanista y social del artista.  
  - **Cluster expresivo:** `hee – hoo – dah – je – na – dirty` agrupa las onomatopeyas y expresiones vocales típicas de su estilo escénico.  
- Estos grupos se alinean con las dimensiones centrales del corpus: **amor, baile, reflexión y performance**.


- **CBOW** ofrece una visión general del vocabulario y conserva las relaciones comunes.  
- **Skip-Gram** separa mejor los temas y capta matices contextuales, mostrando una semántica más rica.  
- Ambos modelos confirman que los embeddings capturan correctamente las **relaciones lingüísticas y estilísticas** de las letras de Michael Jackson.
